In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler,LabelEncoder

import torch.nn.functional as F # Sets of functions such as ReLU
from torchvision import datasets, transforms 

In [190]:
coral=pd.read_csv('./coral_cleaned.csv')
coral=coral.drop('Sample_ID', axis=1)
#coral=coral.set_index('Date_Year')
coral.tail(5)

,Cyclone_Frequency,Depth_m,ClimSST,Ocean_Name,Country_Name,Distance_to_Shore,Exposure,Turbidity,Date_Year,Bleaching_Level,Temperature_Maximum,SSTA,TSA,Percent_Bleaching,Temperature_Mean,Realm_Name,Percent_Cover
34510,85.57,3.0,300.97,Atlantic,United States,49.16,Sheltered,0.0586,2005,Population,303.67,0.32,0.21,100.0,300.34,Tropical Atlantic,28.75
34511,85.57,3.0,300.97,Atlantic,United States,49.16,Sheltered,0.0586,2005,Population,303.67,0.32,0.21,100.0,300.34,Tropical Atlantic,5.62
34512,35.71,14.0,301.58,Indian,Madagascar,8768.03,Sometimes,0.0628,2016,Colony,305.32,0.63,0.59,100.0,301.26,Western Indo-Pacific,19.06
34513,58.42,4.0,299.79,Atlantic,United States,8170.00,Exposed,0.1203,2015,Colony,306.04,3.91,3.91,100.0,299.79,Tropical Atlantic,12.50
34514,62.54,4.0,298.32,Atlantic,United States,1863.00,Sheltered,0.1703,2015,Colony,306.82,1.59,-0.57,100.0,299.65,Tropical Atlantic,12.50


In [191]:
coral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34515 entries, 0 to 34514
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cyclone_Frequency    34515 non-null  float64
 1   Depth_m              34515 non-null  float64
 2   ClimSST              34515 non-null  float64
 3   Ocean_Name           34515 non-null  object 
 4   Country_Name         34515 non-null  object 
 5   Distance_to_Shore    34515 non-null  float64
 6   Exposure             34515 non-null  object 
 7   Turbidity            34515 non-null  float64
 8   Date_Year            34515 non-null  int64  
 9   Bleaching_Level      34515 non-null  object 
 10  Temperature_Maximum  34515 non-null  float64
 11  SSTA                 34515 non-null  float64
 12  TSA                  34515 non-null  float64
 13  Percent_Bleaching    34515 non-null  float64
 14  Temperature_Mean     34515 non-null  float64
 15  Realm_Name           34515 non-null 

In [192]:
coral['Exposure'].value_counts()


Exposure
Sheltered    19504
Exposed      12237
Sometimes     2774
Name: count, dtype: int64

In [193]:
from sklearn.impute import KNNImputer
def Knn_impute(df):
    knn_imputer = KNNImputer(n_neighbors=2)
    coral_numeric = df.copy()
    for column in coral_numeric.select_dtypes(include=['object']).columns:
        coral_numeric[column] = coral_numeric[column].astype('category').cat.codes
    #coral_numeric['Exposure'] = coral_numeric['Exposure'].astype('category').cat.codes
    coral_imputed = knn_imputer.fit_transform(coral_numeric)
    
    coral_imputed = pd.DataFrame(coral_imputed, columns=coral.columns)
    coral_imputed['Exposure'] = pd.Categorical.from_codes(coral_imputed['Exposure'].astype(int), coral['Exposure'].astype('category').cat.categories)

    return coral_imputed  

coral_imputed=Knn_impute(coral)
coral.isna().sum()

Cyclone_Frequency      0
Depth_m                0
ClimSST                0
Ocean_Name             0
Country_Name           0
Distance_to_Shore      0
Exposure               0
Turbidity              0
Date_Year              0
Bleaching_Level        0
Temperature_Maximum    0
SSTA                   0
TSA                    0
Percent_Bleaching      0
Temperature_Mean       0
Realm_Name             0
Percent_Cover          0
dtype: int64

In [194]:
#pd.get_dummies(coral['Coral'], drop_first=True)
#coral_cleaned=coral[coral['Percent_Cover']!=0]

#iterative_imputer = IterativeImputer()
#coral_cleaned=iterative_imputer.fit_transform(coral)
coral_cleaned=coral.copy()
coral_cleaned.reset_index(drop=True, inplace=True)

coral_cleaned.describe()


,Cyclone_Frequency,Depth_m,ClimSST,Distance_to_Shore,Turbidity,Date_Year,Temperature_Maximum,SSTA,TSA,Percent_Bleaching,Temperature_Mean,Percent_Cover
count,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000,34515.000000
mean,52.325062,7.001455,294.191764,3671.828392,0.075548,2008.837317,305.133870,0.279616,-0.960571,9.619465,300.418421,17.212724
std,7.597484,4.135800,14.777580,13384.132023,0.061992,5.719490,1.301263,0.829914,1.645223,20.190956,1.560122,17.373931
min,18.310000,0.000000,262.150000,3.200000,0.000000,1980.000000,300.130000,-4.260000,-11.970000,0.000000,290.880000,0.000000
25%,47.940000,4.000000,298.580000,126.240000,0.039600,2005.000000,304.420000,-0.210000,-1.810000,0.000000,299.770000,5.620000
50%,51.380000,6.000000,300.800000,476.030000,0.057000,2008.000000,305.100000,0.260000,-0.710000,0.250000,300.780000,12.500000
75%,56.060000,10.000000,301.990000,1836.530000,0.084200,2014.000000,305.780000,0.770000,0.120000,6.000000,301.585000,21.250000
max,105.800000,50.300000,307.220000,299218.500000,1.284500,2020.000000,313.140000,5.900000,5.900000,100.000000,303.520000,98.750000


In [195]:
get_dummies_columns=["Ocean_Name", "Country_Name", "Percent_Cover", "Bleaching_Level", "Realm_Name"]
#pd.get_dummies(coral_cleaned['Coral'], drop_first=True)
hot_encoder=OneHotEncoder(sparse_output=False)
coral_hoted=hot_encoder.fit_transform(coral_cleaned[get_dummies_columns])

print("Shape of coral_hoted:", coral_hoted.shape)
encoded_feature_names = hot_encoder.get_feature_names_out(get_dummies_columns)

Shape of coral_hoted: (34515, 452)


In [196]:
if coral_hoted.shape[1] == len(encoded_feature_names):
    # Convert the encoded data to a DataFrame
    encoded_df = pd.DataFrame(coral_hoted, columns=encoded_feature_names)
    print(encoded_df.shape)
else:
    print("Mismatch between the shape of encoded data and the number of feature names.")


df_without_get_dummies = coral_cleaned.drop(columns=get_dummies_columns)

df_concatenated=pd.concat([df_without_get_dummies, encoded_df], axis=1)
df_concatenated.shape

exp_less=df_concatenated.drop('Exposure', axis=1)
le=LabelEncoder()
df_concatenated['Exposure']=le.fit_transform(df_concatenated['Exposure'])

df_concatenated['Exposure'].value_counts()
#le.classes_

(34515, 452)


Exposure
1    19504
0    12237
2     2774
Name: count, dtype: int64

In [197]:

min_max_scaler=MinMaxScaler()
df=pd.DataFrame(min_max_scaler.fit_transform(exp_less), columns=exp_less.columns)

df=pd.concat([df, df_concatenated['Exposure']], axis=1)
df

,Cyclone_Frequency,Depth_m,ClimSST,Distance_to_Shore,Turbidity,Date_Year,Temperature_Maximum,SSTA,TSA,Percent_Bleaching,...,Bleaching_Level_Population,Realm_Name_Central Indo-Pacific,Realm_Name_Eastern Indo-Pacific,Realm_Name_Temperate Australasia,Realm_Name_Temperate Northern Atlantic,Realm_Name_Temperate Northern Pacific,Realm_Name_Tropical Atlantic,Realm_Name_Tropical Eastern Pacific,Realm_Name_Western Indo-Pacific,Exposure
0,0.361070,0.198807,0.875527,0.028461,0.022343,0.625,0.350500,0.374016,0.625070,0.502,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,0.375929,0.278330,0.000000,0.004774,0.020397,0.275,0.375096,0.546260,0.742026,0.507,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.493885,0.139165,0.812958,0.000599,0.033398,0.650,0.308224,0.423228,0.522104,0.509,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
3,0.538119,0.179324,0.843355,0.001036,0.033009,0.650,0.302844,0.412402,0.542809,0.509,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,0.538119,0.248509,0.843133,0.002636,0.033009,0.650,0.279016,0.419291,0.547286,0.509,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,0.768774,0.059642,0.861327,0.000154,0.045621,0.625,0.272098,0.450787,0.681589,1.000,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
34511,0.768774,0.059642,0.861327,0.000154,0.045621,0.625,0.272098,0.450787,0.681589,1.000,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
34512,0.198880,0.278330,0.874861,0.029293,0.048891,0.900,0.398924,0.481299,0.702854,1.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
34513,0.458452,0.079523,0.835145,0.027294,0.093655,0.875,0.454266,0.804134,0.888640,1.000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0


In [198]:
#df=df.reindex(columns=[col for col in df.columns if col != 'Exposure'] + ['Exposure'])
#df.head()


In [199]:
class CoralExposureDataset(Dataset):
    def __init__(self,data,transform=None):
        super().__init__()
        self.transform=transform
        self.data=data
    
    def __getitem__(self,idx):
        row=self.data.iloc[idx]
        feature = row[:-1].values.astype('float32')
        target=row[-1]
        if self.transform:
            feature=self.transform(feature)
        return torch.tensor(feature,dtype=torch.float32), torch.tensor(target,dtype=torch.float32)
        
    def __len__(self):
        return len(self.data)
    

In [200]:
dataset=CoralExposureDataset(df)

train_size=int(0.8*len(dataset))
test_size=len(dataset)-train_size

train_data,test_data=torch.utils.data.random_split(dataset,[train_size,test_size])


train_data_loader=DataLoader(train_data,batch_size=32,shuffle=True)
test_data_loader=DataLoader(test_data,batch_size=32,shuffle=True)

In [201]:
class CoralExposureClassifier(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(CoralExposureClassifier, self).__init__()
        self.linear1=nn.Linear(input_dim,64)
        self.batchnorm1=nn.BatchNorm1d(64)
        self.linear2=nn.Linear(64,32)
        self.batchnorm2=nn.BatchNorm1d(32)
        self.linear3=nn.Linear(32,16)
        self.batchnorm3=nn.BatchNorm1d(16)
        self.linear4=nn.Linear(16,1)
    
    def forward(self,x):
        x=F.relu(self.batchnorm1(self.linear1(x)))
        x=F.relu(self.batchnorm2(self.linear2(x)))
        x=F.relu(self.batchnorm3(self.linear3(x)))
        x=self.linear4(x)
        x=F.softmax(x,dim=1)
        return x

input_dim=df.shape[1]-1
output_dim=3
model = CoralExposureClassifier(input_dim, output_dim)

In [202]:
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)    

In [203]:
epoch_nums=70
train_losses = []
model.train()
for epoch in range(epoch_nums):
    total_train_loss = 0
    for data,target in train_data_loader:
        optimizer.zero_grad()
        output=model(data)
        loss=criterion(output,target.view(-1,1))
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        
    avg_train_loss = total_train_loss / len(train_data_loader)
    train_losses.append(avg_train_loss)    
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',loss.item())

        

/var/folders/vc/3kfwh15d7234657gp2jqxqj80000gn/T/ipykernel_15228/3294585815.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target=row[-1]


Epoch: 0 Loss: 0.5714285969734192
Epoch: 10 Loss: 0.5357142686843872
Epoch: 20 Loss: 0.3571428656578064
Epoch: 30 Loss: 0.5714285969734192
Epoch: 40 Loss: 0.3928571343421936
Epoch: 50 Loss: 0.5
Epoch: 60 Loss: 0.4642857015132904


In [204]:

val_losses = []
model.eval()
with torch.no_grad():
    total_loss=0
    for i,(data,target) in enumerate(test_data_loader):
        output=model(data)
        loss=criterion(output,target)
        total_loss += loss.item()
    avg_val_loss=total_loss/len(test_data_loader)
    val_losses.append(avg_val_loss)
    print('Validation Loss:',np.mean(val_losses))
    
    

/var/folders/vc/3kfwh15d7234657gp2jqxqj80000gn/T/ipykernel_15228/3294585815.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target=row[-1]
/Users/Mehr/Desktop/Python/me/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Validation Loss: 0.4308071658015251


/Users/Mehr/Desktop/Python/me/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([23])) that is different to the input size (torch.Size([23, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [206]:
import optuna


class CoralModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2):
        super(CoralModel, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim1)
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim1)
        self.linear2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.batchnorm2 = nn.BatchNorm1d(hidden_dim2)
        self.linear3 = nn.Linear(hidden_dim2, 1)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = F.relu(self.batchnorm1(self.linear1(x)))
        x = self.dropout(x)
        x = F.relu(self.batchnorm2(self.linear2(x)))
        x = self.linear3(x)
        return x
    
def objective(trial):
    input_dim = df.shape[1] - 1
    
    hidden_dim1 = trial.suggest_int('hidden_dim1', 64, 256)
    hidden_dim2 = trial.suggest_int('hidden_dim2', 16, 128)
    
    model = CoralModel(input_dim, hidden_dim1, hidden_dim2)
    
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 16, 128, log=True)
    

    criterion = nn.MSELoss()
    num_epochs=50
    train_losses = []
    model.train()
    for epoch in range(num_epochs):
        total_train_loss = 0
        for feature, target in train_data_loader:
            optimizer.zero_grad()
            outputs = model(feature)
            loss = criterion(outputs, target.view(-1, 1))
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(train_data_loader)
        train_losses.append(avg_train_loss)
    
    model.eval()
    val_losses = []
    with torch.no_grad():
        total_loss = 0
        for feature, target in test_data_loader:
            outputs = model(feature)
            loss = criterion(outputs, target.view(-1, 1))
            total_loss += loss.item()
        avg_val_loss = total_loss / len(test_data_loader)
        val_losses.append(avg_val_loss)
    
    return avg_val_loss


study=optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=15)

print('Number of finished trials:', len(study.trials))
print('Best trial params:', study.best_trial.params)

[I 2024-07-11 02:56:34,601] A new study created in memory with name: no-name-898cf924-2afe-49f3-a7e1-a91a365efb22
/var/folders/vc/3kfwh15d7234657gp2jqxqj80000gn/T/ipykernel_15228/3777389925.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/var/folders/vc/3kfwh15d7234657gp2jqxqj80000gn/T/ipykernel_15228/3294585815.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target=row[-1]
[I 2024-07-11 03:08:58,319] Trial 0 finished with value: 0.8950258181602867 and parameters: {'hidden_dim1': 242, 'hidden_dim2': 63, 'lr': 0.0032446830711106898, 'batch_size': 44}. Best is trial 0 with value: 0.895025

KeyboardInterrupt: 

In [207]:
dataset=CoralExposureDataset(df)

train_size=int(0.8*len(dataset))
test_size=len(dataset)-train_size

train_data,test_data=torch.utils.data.random_split(dataset,[train_size,test_size])


train_data_loader=DataLoader(train_data,batch_size=29,shuffle=True)
test_data_loader=DataLoader(test_data,batch_size=29,shuffle=True)

class CoralExposureClassifier(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(CoralExposureClassifier, self).__init__()
        self.linear1=nn.Linear(input_dim,171)
        self.batchnorm1=nn.BatchNorm1d(171)
        self.linear2=nn.Linear(171,105)
        self.batchnorm2=nn.BatchNorm1d(105)
        self.linear3=nn.Linear(105,16)
        self.batchnorm3=nn.BatchNorm1d(16)
        self.linear4=nn.Linear(16,1)
    
    def forward(self,x):
        x=F.relu(self.batchnorm1(self.linear1(x)))
        x=F.relu(self.batchnorm2(self.linear2(x)))
        x=F.relu(self.batchnorm3(self.linear3(x)))
        x=self.linear4(x)
        x=F.softmax(x,dim=1)
        return x

input_dim=df.shape[1]-1
output_dim=3
model = CoralExposureClassifier(input_dim, output_dim)

#[I 2024-07-11 03:31:53,534] Trial 2 finished with value: 0.7282506233012235 and parameters: {'hidden_dim1': 171, 'hidden_dim2': 105, 'lr': 0.006309289907589171, 'batch_size': 29}. Best is trial 2 with value: 0.7282506233012235.
optimizer=optim.Adam(model.parameters(),lr=0.006309289907589171)
criterion=nn.MSELoss()
epoch_nums=70
train_losses = []
model.train()
for epoch in range(epoch_nums):
    total_train_loss = 0
    for data,target in train_data_loader:
        optimizer.zero_grad()
        output=model(data)
        loss=criterion(output,target.view(-1,1))
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        
    avg_train_loss = total_train_loss / len(train_data_loader)
    train_losses.append(avg_train_loss)    
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',loss.item())



val_losses = []
model.eval()
with torch.no_grad():
    total_loss=0
    for i,(data,target) in enumerate(test_data_loader):
        output=model(data)
        loss=criterion(output,target)
        total_loss += loss.item()
    avg_val_loss=total_loss/len(test_data_loader)
    val_losses.append(avg_val_loss)
    print('Validation Loss:',np.mean(val_losses))

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()
    

/var/folders/vc/3kfwh15d7234657gp2jqxqj80000gn/T/ipykernel_15228/3294585815.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target=row[-1]


In [ ]:

torch.save(model.state_dict(), 'coral_model.pth')


input_dim = df.shape[1] - 1
hidden_dim1 = study.best_trial.params['hidden_dim1']
hidden_dim2 = study.best_trial.params['hidden_dim2']
model = CoralModel(input_dim, hidden_dim1, hidden_dim2)
model.load_state_dict(torch.load('coral_model.pth'))
model.eval()



with torch.no_grad():
    predictions = model(new_data)